In [1]:
import json
from openai import OpenAI
from dotenv import load_dotenv
import os
from openai import OpenAI
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY")) # Get Key from Env (ignored in git) 
client = OpenAI()
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')
from tqdm.auto import tqdm
import fitz  # PyMuPDF
## think i dont need any more ## from pathlib import Path

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set WD
import os 
os.getcwd()
# change path to the ma-rag-pipeline directory 
os.chdir("/workspaces/llm-zoomcamp/ma-rag-pipeline")

### 1. Import new entity and utility functions. 

In [3]:
from src.entities.user import load_users, get_user_by_id
from src.entities.spiritual_profile import load_spiritual_profiles, get_profile_by_user_id
from src.entities.conversation import load_conversations, append_conversation
from src.entities.persona import load_personas, get_persona_by_id
from src.utils.io import load_json, save_json, append_json_record
from pathlib import Path # I was already using this 

### 2. Set Paths

In [4]:

user_path = Path("data/users.json")
persona_path = Path("data/personas.json")
spiritual_profile_path = Path("data/spiritual_profiles.json")
conversation_path = Path("data/conversations.json")

### 3. Load Data

In [5]:
users = load_users(user_path)
spiritual_profiles = load_spiritual_profiles(spiritual_profile_path)
personas = load_personas(persona_path)
conversations = load_conversations(conversation_path)

### 4. Select Persona

### 5. initialize current conversation history

In [ ]:
current_conversation_history = []

In [40]:
# RAG and Prompt Functions
def build_initial_conversation_prompt(query, conversation_history=None):
    """Build prompt for initial conversation using spiritual_explorer persona from JSON"""
    
    # Get spiritual_explorer persona from file
    persona = get_persona_by_id("spiritual_explorer", personas).to_dict() # Pull the beginner persona from the personas.json file
    
    # Build conversation context if history exists
    history_text = ""
    if conversation_history:
        recent_history = conversation_history[-4:]  # Last 2 exchanges
        for msg in recent_history:
            role = "user" if msg["role"] == "user" else "sponsor"
            history_text += f"{role}: {msg['content']}\n"
        history_text = f"\nRecent conversation:\n{history_text}"
    
    prompt_template = f"""You are an experienced MA sponsor meeting someone for their very first time at Marijuana Anonymous.

Your communication style: {persona['language_style']} 
Your approach: {persona['description']}

This is an initial meeting conversation. Your goals:
1. Welcome them warmly and create a safe space
2. Learn their name
3. Understand what brought them to MA today  
4. Gently assess their relationship with marijuana
5. Begin to understand their spiritual openness (without being pushy)
6. Determine if they're ready to start recovery work

Keep responses conversational and warm (2-3 sentences max). Don't rush through questions - let the conversation flow naturally.{history_text}

User says: "{query}"

Respond as their sponsor:"""
    
    return prompt_template

In [41]:
def llm_initial(prompt, model="gpt-4o"):
    """LLM call for initial conversations"""
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a warm, experienced MA sponsor. Keep responses natural and conversational."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

In [42]:
def rag_initial_conversation(query, conversation_history=None):
    """RAG function for initial conversations"""
    prompt = build_initial_conversation_prompt(query, conversation_history)
    answer = llm_initial(prompt)
    # Update conversation history
    current_conversation_history.append({"role": "user", "content": query})
    current_conversation_history.append({"role": "sponsor", "content": answer})
    
    return answer

In [43]:
rag_initial_conversation("Hi, I'm new here and looking for help with my marijuana use.", current_conversation_history)

"Hi there! It's so nice to meet you, and I'm glad you’re here. Can I ask your name?"

In [ ]:
import time
# Main loop for conversation
while True:
    user_query = input("You: ")
    if user_query.strip().lower() in ["bye", "goodbye", "exit", "quit"]:
        print("Sponsor: Goodbye! Take care.")
        break
    sponsor_reply = rag_initial_conversation(user_query, conversation_history=current_conversation_history)
    print(f"Sponsor: {sponsor_reply}")
    time.sleep(0.5)  # Add a half-second pause before next input

Sponsor: Hello there! I'm really glad you decided to join us today. Can I ask what name you'd like to go by here?
Sponsor: Hi Steve, it's really nice to meet you. I'm really glad you decided to join us today. What brought you to Marijuana Anonymous?
Sponsor: I appreciate you sharing that with me, Steve. It sounds like you're facing a significant challenge with the upcoming move. What do you think makes you feel most vulnerable about going back to Canada?
Sponsor: I hear you, Steve. Being in a place where it's so accessible and feeling alone can definitely be tough. Have you thought about any strategies or support systems that might help you stay grounded while you're there?
Sponsor: It's completely okay to reach out to your sponsor, Steve. That's what we're here for—to support each other, especially during challenging times. Maybe we can brainstorm a few more ways to help you feel connected and supported while you're there. How does that sound?
Sponsor: Great, Steve. Let's think about 